# Analyze city challenges to tax-exempt properties

In [1]:
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import geopandas as gpd
import pandas as pd
import requests

In [2]:
db = create_engine('postgresql://jon@localhost:5432/propertydb')

In [3]:
parcels = pd.read_csv('input/pghexempt.csv')

### Load parcel data

In [4]:
parcelpoints = gpd.read_postgis("""SELECT parcelcentroids.parcelid, parcelmbl, taxdesc, localtotal::int, ST_MakePoint(lon::float, lat::float) AS geom
FROM parcelcentroids
JOIN assessments
ON assessments.parcelid = parcelcentroids.parcelid
WHERE parcelmbl IN %(parcels)s;""", db, params={'parcels': tuple(parcels.parcelmbl)}, crs='EPSG:4269')
parcelpoints = parcelpoints.to_crs('EPSG:4326')

In [5]:
parcels = parcelpoints.merge(parcels, on='parcelmbl')

### Set attributes, export for Datawrapper

In [6]:
parcels['title'] = ''
parcels['tooltip'] = '<b>Parcel ID:</b> ' + parcels.parcelmbl + '<br><b>Owner:</b> ' + parcels.owner

parcels['color'] = 'black'
parcels.loc[parcels.owner == 'Allegheny General Hospital', 'color'] = '#008000'
parcels.loc[parcels.owner == 'Carnegie Mellon University', 'color'] = '#FF0000'
parcels.loc[parcels.owner == 'Presbyterian University Health System (UPMC)', 'color'] = '#800080'
parcels.loc[parcels.owner == 'University of Pittsburgh', 'color'] = '#0000FF'

In [7]:
parcels[['title', 'tooltip', 'color', 'geom']].sort_values(by='tooltip').to_file('output/pghexempt.geojson', driver='GeoJSON')

### Check outcome of challenges

In [8]:
pd.read_sql_query("""SELECT DISTINCT(asofdate) FROM assessments;""", db)

,asofdate
0,2023-09-01


##### Get newest now-taxable properties from the county website

In [9]:
for index, row in parcels.iterrows():
    response = requests.get('https://www2.alleghenycounty.us/RealEstate/GeneralInfo.aspx', params={ 'ParcelID': row.parcelid })
    response = BeautifulSoup(response.text, 'html.parser')

    parcels.loc[index, 'PORTALtaxdec'] = response.find('span', id='lblTax').contents[0]

In [10]:
parcels[(parcels.PORTALtaxdec == 'Taxable') & (parcels.taxdesc == '10 - Exempt')]

,parcelid,parcelmbl,taxdesc,localtotal,geom,owner,title,tooltip,color,PORTALtaxdec


##### By tax status, taxable value

In [11]:
summary = parcels.groupby('taxdesc').agg({ 'parcelmbl': 'count', 'localtotal': 'sum' })
summary['city_annualtaxbill'] = (summary.localtotal / 1000) * 8.06
summary['pct_parcelmbl'] = summary.parcelmbl / summary.parcelmbl.sum()
summary['pct_localtotal'] = summary.localtotal / summary.localtotal.sum()

summary

,parcelmbl,localtotal,city_annualtaxbill,pct_parcelmbl,pct_localtotal
taxdesc,,,,,
10 - Exempt,17,64059600,516320.376,0.653846,0.882219
20 - Taxable,9,8552300,68931.538,0.346154,0.117781


##### By owner

In [12]:
pd.pivot_table(parcels, index=['owner', 'taxdesc'], values='parcelmbl', aggfunc='count')

,,parcelmbl
owner,taxdesc,
8033 Bennet Street RE LLC,20 - Taxable,1
Allegheny General Hospital,10 - Exempt,1
Carnegie Mellon University,20 - Taxable,2
Community Options Inc.,10 - Exempt,1
Karpeles Manuscript Library,10 - Exempt,1
Mr. Property LLC,20 - Taxable,1
NCSC USA Housing Development Corp.,10 - Exempt,1
Presbyterian University Health System (UPMC),10 - Exempt,6
Private citizen,10 - Exempt,5
